In [6]:
import pandas as pd
import numpy as np

### Determine the effective spread using the block method `cov([dt1, dt2, dt3], [dt4, dt5, dt6])`

### Determine the effective spread using the serial covariance `cov([dt1, dt2, dt3], [dt2, dt3, dt4])`

In [ ]:
data = pd.read_csv('data/aapl_5m_train.csv')
data.drop(columns=['Unnamed: 0', 'Gmtoffset', 'Datetime', 'Volume'], inplace=True)
data['Diff'] = data['Close'].diff()
data['Diff_Shift'] = data['Diff'].shift()
data = data.dropna()

windows = [5, 10, 20, 50, 100]


for window in windows:
    data[f'W_{window}'] = data['Diff'].rolling(window=window).cov(data['Diff_Shift'])

for i in range(len(windows)):
    data[f'Spread_{windows[i]}'] = 2 * np.sqrt(np.abs(data[f'W_{windows[i]}']))


bid_ask = pd.DataFrame()
for i in range(len(windows)):
    bid_ask[f'Bid_{windows[i]}'] = np.where(data['Diff'] > 0,
                                            data['Close'],
                                            data['Close'] - data[f'Spread_{windows[i]}'])
    bid_ask[f'Ask_{windows[i]}'] = np.where(data['Diff'] > 0,
                                            data['Close'] + data[f'Spread_{windows[i]}'],
                                            data['Close'])
bid_ask.head(30)

,Bid_5,Ask_5,Bid_10,Ask_10,Bid_20,Ask_20,Bid_50,Ask_50,Bid_100,Ask_100
0,132.059997,NaN,132.059997,NaN,132.059997,NaN,132.059997,NaN,132.059997,NaN
1,132.250000,NaN,132.250000,NaN,132.250000,NaN,132.250000,NaN,132.250000,NaN
2,NaN,131.589996,NaN,131.589996,NaN,131.589996,NaN,131.589996,NaN,131.589996
3,NaN,131.448898,NaN,131.448898,NaN,131.448898,NaN,131.448898,NaN,131.448898
4,130.554683,131.119995,NaN,131.119995,NaN,131.119995,NaN,131.119995,NaN,131.119995
5,130.639457,130.875000,NaN,130.875000,NaN,130.875000,NaN,130.875000,NaN,130.875000
6,130.032152,130.509902,NaN,130.509902,NaN,130.509902,NaN,130.509902,NaN,130.509902
7,130.889999,131.159861,130.889999,NaN,130.889999,NaN,130.889999,NaN,130.889999,NaN
8,130.935806,131.088253,130.935806,NaN,130.935806,NaN,130.935806,NaN,130.935806,NaN
9,131.053604,131.270971,131.053604,131.361040,131.053604,NaN,131.053604,NaN,131.053604,NaN
